# ESPCN Realtime Webcam Upscaler - Training Script (Google Colab)

**Project:** ESPCN Realtime Webcam Upscaler  
**Institution:** Universitas Tarumanagara  
**Thesis:** Development Of An ESPCN Deep Learning Model For Real-Time Webcam Video Super-Resolution With Hardware Interference Optimization  
**Platform:** Google Colab Cloud  
**Date:** January 2026

## Environment Specification
- **OS:** Ubuntu 22.04.4 LTS (Colab Runtime)
- **Python:** 3.10+
- **CPU:** Intel Xeon E5-2699 v4 (1 Core / 2 Thread shared)
- **RAM:** 13GB (shared)
- **GPU:** NVIDIA T4 (optional, free tier) or higher
- **Framework:** PyTorch 2.x with CUDA support

## Overview
This notebook implements ESPCN training in Google Colab with cloud storage integration. The pipeline:
- Mounts Google Drive for persistent storage
- Auto-configures batch size based on available GPU
- Supports training on free T4 GPU or CPU fallback
- Saves checkpoints and logs to Drive
- Performs model quantization and ONNX export

## 1. Library Imports

Import all required libraries for training and cloud integration.

In [ ]:
# Standard library imports
import os
import time
import math
import glob
import random
from pathlib import Path

# Third-party imports
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.amp import GradScaler, autocast
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision.transforms import ToTensor, ToPILImage, Compose, ColorJitter

# Progress bar and utilities
from tqdm import tqdm

# ONNX and inference
import onnxruntime as ort
from pytorch_msssim import ssim

# Google Drive integration (Colab-specific)
from google.colab import drive

## 2. Colab Environment Setup

Install dependencies and mount Google Drive for persistent storage.

In [ ]:
# Install required packages
print("Installing dependencies...")
get_ipython().run_line_magic('pip', 'install pytorch-msssim onnxruntime-gpu -q')
print("✓ Dependencies installed\n")

# Import ssim after installation
from pytorch_msssim import ssim

# ========== MOUNT GOOGLE DRIVE ==========
if not os.path.exists('/content/drive/MyDrive'):
    print("📁 Mounting Google Drive...")
    drive.mount('/content/drive')
    print("✓ Google Drive mounted\n")
else:
    print("✓ Google Drive already mounted\n")

# ========== COPY DATASETS FROM DRIVE ==========
if not os.path.exists('/content/datasets'):
    print("📋 Copying datasets from Google Drive...")
    print("  Note: This may take 2-5 minutes depending on dataset size")
    os.system('sudo cp -rf /content/drive/MyDrive/Thesis/datasets /content/datasets')
    print("✓ Datasets copied\n")
else:
    print("✓ Datasets already available locally\n")

# ========== AUTO-CONFIGURE BATCH SIZE ==========
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

if DEVICE == "cuda":
    gpu_name = torch.cuda.get_device_name(0)
    print(f"🎮 GPU detected: {gpu_name}")
    
    if "T4" in gpu_name:
        BATCH_SIZE = 128
        print("   → Using T4 GPU with BATCH_SIZE=128")
    elif "V100" in gpu_name or "A100" in gpu_name:
        BATCH_SIZE = 256
        print("   → Using high-end GPU with BATCH_SIZE=256")
    else:
        BATCH_SIZE = 64
        print(f"   → Using {gpu_name} with BATCH_SIZE=64")
else:
    BATCH_SIZE = 64
    print("💻 CPU-only mode detected")
    print("   → Using BATCH_SIZE=64 (slower but works)\n")

## 3. Training Configuration & Hyperparameters

Configure model architecture and training schedule.

In [ ]:
# ========================= CONFIGURATION =========================
# Model Architecture
UPSCALE_FACTOR = 2  # 2x upscaling

# Training Hyperparameters
EPOCHS = 150  # Total training epochs
PATCH_SIZE = 256  # Training patch size
LEARNING_RATE = 1e-4  # Adam optimizer learning rate
LOSS_ALPHA = 0.84  # Weight for MSE loss

# Hardware Configuration (BATCH_SIZE set above based on GPU)
NUM_WORKER = os.cpu_count()  # Use all available CPU cores
print(f"🔧 Configuration:")
print(f"   Upscale Factor: {UPSCALE_FACTOR}x")
print(f"   Batch Size: {BATCH_SIZE}")
print(f"   Epochs: {EPOCHS}")
print(f"   Learning Rate: {LEARNING_RATE:.0e}")
print(f"   Loss Alpha (MSE weight): {LOSS_ALPHA}")
print(f"   DataLoader Workers: {NUM_WORKER}\n")

# Directory Structure (using Google Drive paths for persistence)
BASE_DIR = Path("/content/drive/MyDrive/Thesis")
DATA_DIR = Path("/content/datasets")
OUTPUTS_DIR = BASE_DIR / "outputs"
CHECKPOINTS_DIR = BASE_DIR / "checkpoints"
CHECKPOINT_FILE = CHECKPOINTS_DIR / f"best_espcn_x{UPSCALE_FACTOR}.pth"

# ========================= REPRODUCIBILITY =========================
def set_seed(seed=42):
    """Set random seeds for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


def seed_worker(worker_id):
    """Set seed for DataLoader worker processes."""
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)


set_seed()

## 4. Dataset Preparation

In [ ]:
def prepare_environment_and_datasets(patch_size):
    """Prepare training environment and validate datasets."""
    CHECKPOINTS_DIR.mkdir(exist_ok=True, parents=True)
    OUTPUTS_DIR.mkdir(exist_ok=True, parents=True)

    CACHE_FILE_TRAIN = OUTPUTS_DIR / "valid_train_paths.txt"

    # Validation dataset
    valid_div2k_dir = DATA_DIR / "DIV2K_valid_HR"
    validation_image_paths = glob.glob(str(valid_div2k_dir / '*.*'))

    # Training dataset (with caching)
    valid_train_paths = []
    if CACHE_FILE_TRAIN.exists():
        print(f"Loading cached training paths...")
        try:
            with open(CACHE_FILE_TRAIN, 'r') as f:
                valid_train_paths = [line.strip() for line in f if line.strip()]
            print(f"✓ Loaded {len(valid_train_paths)} paths")
        except Exception as e:
            print(f"⚠ Cache error: {e}. Rebuilding...")
            valid_train_paths = []

    if not valid_train_paths:
        print(f"Building training image list...")
        train_div2k_dir = DATA_DIR / "DIV2K_train_HR"
        train_flickr2k_dir = DATA_DIR / "Flickr2K_HR"
        personal_dir = DATA_DIR / "Personal_HR"

        train_image_paths = glob.glob(str(train_div2k_dir / '*.*')) + \
                            glob.glob(str(train_flickr2k_dir / '*.*'))

        if personal_dir.exists():
            print("✓ Personal dataset found")
            train_image_paths += glob.glob(str(personal_dir / '*.*'))

        print(f"Found {len(train_image_paths)} candidate images")

        def is_image_large_enough(image_path, min_size):
            try:
                img = cv2.imread(str(image_path))
                if img is None:
                    return False
                h, w = img.shape[:2]
                return h >= min_size and w >= min_size
            except:
                return False

        valid_train_paths = [
            p for p in tqdm(train_image_paths, desc="Filtering")
            if is_image_large_enough(p, patch_size)
        ]

        print(f"✓ Filtered: {len(valid_train_paths)}/{len(train_image_paths)} valid")

        try:
            with open(CACHE_FILE_TRAIN, 'w') as f:
                for path in valid_train_paths:
                    f.write(f"{path}\n")
        except Exception as e:
            print(f"⚠ Cache write failed: {e}")

    if not valid_train_paths or not validation_image_paths:
        raise RuntimeError("No datasets found. Check directory structure.")

    return valid_train_paths, validation_image_paths

## 5. Loss Function

In [ ]:
class CombinedLoss(nn.Module):
    """Combined MSE + SSIM loss for super-resolution."""
    def __init__(self, alpha=0.7):
        super(CombinedLoss, self).__init__()
        self.alpha = alpha
        self.mse_loss = nn.MSELoss()
        self.ssim_loss_fn = ssim

    def forward(self, output, target):
        loss_mse = self.mse_loss(output, target)
        ssim_score = self.ssim_loss_fn(output, target, data_range=1.0, size_average=True)
        loss_ssim = 1 - ssim_score
        total_loss = self.alpha * loss_mse + (1 - self.alpha) * loss_ssim
        return total_loss, loss_mse, ssim_score


def psnr(mse):
    """Calculate PSNR from MSE."""
    return 10 * math.log10(1 / mse) if mse > 0 else float('inf')

## 6. Dataset Classes

In [ ]:
class TrainingSuperResolutionDataset(Dataset):
    """Training dataset with online augmentation."""
    def __init__(self, image_filenames, crop_size, upscale_factor):
        super().__init__()
        self.image_filenames = image_filenames
        self.crop_size = crop_size - (crop_size % upscale_factor)
        self.upscale_factor = upscale_factor
        self.to_tensor = ToTensor()
        self.scale_factor = 1 / upscale_factor
        self.transform = Compose([
            ToPILImage(),
            ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
            ToTensor()
        ])

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, index):
        try:
            hr_image = cv2.imread(self.image_filenames[index])
            if hr_image is None:
                raise IOError(f"Failed to load image")
            hr_image = cv2.cvtColor(hr_image, cv2.COLOR_BGR2RGB)
        except Exception as e:
            return self.__getitem__((index + 1) % len(self))

        h, w = hr_image.shape[:2]
        i = random.randint(0, h - self.crop_size)
        j = random.randint(0, w - self.crop_size)
        hr_patch = hr_image[i:i+self.crop_size, j:j+self.crop_size, :]

        if torch.rand(1) > 0.5:
            hr_patch = cv2.flip(hr_patch, 1)
        if torch.rand(1) > 0.5:
            angle = random.choice([cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_180, cv2.ROTATE_90_COUNTERCLOCKWISE])
            hr_patch = cv2.rotate(hr_patch, angle)

        hr_patch_tensor = self.transform(hr_patch)
        lr_patch_tensor = F.interpolate(
            hr_patch_tensor.unsqueeze(0), scale_factor=self.scale_factor,
            mode='bicubic', align_corners=False, antialias=True
        ).squeeze(0)

        return lr_patch_tensor, hr_patch_tensor


class ValidationSuperResolutionDataset(Dataset):
    """Validation dataset without augmentation."""
    def __init__(self, image_filenames, upscale_factor):
        super().__init__()
        self.image_filenames = image_filenames
        self.upscale_factor = upscale_factor
        self.to_tensor = ToTensor()
        self.scale_factor = 1 / upscale_factor

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, index):
        try:
            hr_image = cv2.imread(self.image_filenames[index])
            if hr_image is None:
                raise IOError()
            hr_image = cv2.cvtColor(hr_image, cv2.COLOR_BGR2RGB)
        except:
            return self.__getitem__((index + 1) % len(self))

        h, w = hr_image.shape[:2]
        w_new, h_new = w - (w % self.upscale_factor), h - (h % self.upscale_factor)
        hr_image = hr_image[:h_new, :w_new, :]

        hr_tensor = self.to_tensor(hr_image)
        lr_tensor = F.interpolate(
            hr_tensor.unsqueeze(0), scale_factor=self.scale_factor,
            mode='bicubic', align_corners=False, antialias=True
        ).squeeze(0)

        return lr_tensor, hr_tensor

## 7. Model Architecture

In [ ]:
class ESPCN(nn.Module):
    """Efficient Sub-Pixel Convolutional Neural Network."""
    def __init__(self, upscale_factor):
        super(ESPCN, self).__init__()
        self.upscale_factor = upscale_factor
        self.conv1 = nn.Conv2d(3, 64, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 3 * (upscale_factor ** 2), kernel_size=3, padding=1)
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.pixel_shuffle(self.conv3(x))
        return x

## 8. Training Execution

In [ ]:
if __name__ == "__main__":
    print("\n" + "="*70)
    print("ESPCN TRAINING - GOOGLE COLAB")
    print("="*70)
    
    print(f"\n📊 Device: {DEVICE}")
    if DEVICE == "cuda":
        print(f"   GPU: {torch.cuda.get_device_name(0)}")
        print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")

    # Prepare datasets
    print(f"\n📁 Preparing datasets...")
    train_paths, val_paths = prepare_environment_and_datasets(PATCH_SIZE)
    
    if not train_paths or not val_paths:
        raise RuntimeError("Dataset preparation failed")

    # Create datasets
    train_dataset = TrainingSuperResolutionDataset(train_paths, crop_size=PATCH_SIZE, upscale_factor=UPSCALE_FACTOR)
    val_dataset = ValidationSuperResolutionDataset(val_paths, upscale_factor=UPSCALE_FACTOR)

    # Create dataloaders
    train_loader = DataLoader(
        train_dataset, batch_size=BATCH_SIZE, shuffle=True,
        num_workers=min(4, NUM_WORKER),
        pin_memory=(DEVICE == "cuda"),
        worker_init_fn=seed_worker
    )
    val_loader = DataLoader(
        val_dataset, batch_size=1, shuffle=False,
        num_workers=min(4, NUM_WORKER),
        pin_memory=(DEVICE == "cuda")
    )

    print(f"✓ Training samples: {len(train_dataset)}")
    print(f"✓ Validation samples: {len(val_dataset)}")

    # Initialize model
    model = ESPCN(upscale_factor=UPSCALE_FACTOR).to(DEVICE)
    if DEVICE == 'cuda' and torch.cuda.device_count() > 1:
        model = torch.nn.DataParallel(model)
    
    num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"\n🧠 Model: ESPCN x{UPSCALE_FACTOR} ({num_params:,} parameters)")

    # Initialize training components
    criterion = CombinedLoss(alpha=LOSS_ALPHA).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
    scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)
    scaler = GradScaler(enabled=(DEVICE == "cuda"))

    # Setup checkpointing
    start_epoch = 0
    best_psnr = 0.0
    log_file_path = OUTPUTS_DIR / 'training_log.csv'
    training_log = []

    if CHECKPOINT_FILE.exists():
        print(f"\n↻ Loading checkpoint...")
        checkpoint = torch.load(CHECKPOINT_FILE, map_location=DEVICE)
        original_state_dict = checkpoint['model_state_dict']
        new_state_dict = {}

        needs_prefix_stripping = any(key.startswith('_orig_mod.') for key in original_state_dict.keys())
        if needs_prefix_stripping:
            for key, value in original_state_dict.items():
                new_key = key[len('_orig_mod.'):] if key.startswith('_orig_mod.') else key
                new_state_dict[new_key] = value
        else:
            new_state_dict = original_state_dict

        needs_module_stripping = any(key.startswith('module.') for key in new_state_dict.keys())
        if needs_module_stripping:
            stripped_dict = {}
            for key, value in new_state_dict.items():
                new_key = key[len('module.'):] if key.startswith('module.') else key
                stripped_dict[new_key] = value
            new_state_dict = stripped_dict

        model.load_state_dict(new_state_dict)
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        best_psnr = checkpoint.get('best_psnr', 0.0)
        print(f"   ✓ Resumed from epoch {start_epoch} (best PSNR: {best_psnr:.4f} dB)")

        if log_file_path.exists():
            try:
                log_df = pd.read_csv(log_file_path)
                log_df = log_df[log_df['epoch'] < start_epoch]
                training_log = log_df.to_dict('records')
            except:
                training_log = []
    else:
        print(f"\n▶ Starting training from scratch")
        if log_file_path.exists():
            log_file_path.unlink()

    # Enable torch.compile if available
    try:
        print(f"Compiling model with torch.compile...")
        model = torch.compile(model)
    except Exception as e:
        print(f"⚠ torch.compile failed: {e}. Using eager mode.")

    # Begin training
    print(f"\n{'='*70}")
    print(f"TRAINING ESPCN x{UPSCALE_FACTOR} | Epochs {start_epoch+1}-{EPOCHS}")
    print(f"{'='*70}\n")

    start_time = time.time()
    for epoch in range(start_epoch, EPOCHS):
        model.train()

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")
        for lr_images, hr_images in progress_bar:
            lr_images = lr_images.to(DEVICE, non_blocking=(DEVICE == "cuda"))
            hr_images = hr_images.to(DEVICE, non_blocking=(DEVICE == "cuda"))

            optimizer.zero_grad()

            if DEVICE == "cuda":
                with autocast(device_type=DEVICE):
                    outputs = model(lr_images)
                    total_loss, mse, ssim_score = criterion(outputs.float(), hr_images.float())
                scaler.scale(total_loss).backward()
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = model(lr_images)
                total_loss, mse, ssim_score = criterion(outputs.float(), hr_images.float())
                total_loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

            progress_bar.set_postfix(
                Loss=f"{total_loss.item():.4f}",
                SSIM=f"{ssim_score.item():.4f}"
            )

        # Validation
        model.eval()
        val_psnr, val_ssim = 0.0, 0.0
        with torch.no_grad():
            for i, (lr_images, hr_images) in enumerate(val_loader):
                lr_images = lr_images.to(DEVICE)
                hr_images = hr_images.to(DEVICE)
                output = model(lr_images)
                mse_val = nn.functional.mse_loss(output, hr_images)
                val_psnr += psnr(mse_val.item())
                val_ssim += ssim(output, hr_images, data_range=1.0, size_average=True).item()

                if i == 0 and (epoch + 1) % 10 == 0:
                    sr_pil = ToPILImage()(output.squeeze(0).cpu())
                    output_path = OUTPUTS_DIR / f'val_epoch_{epoch+1:03d}.png'
                    sr_pil.save(output_path)

        avg_val_psnr = val_psnr / len(val_loader)
        avg_val_ssim = val_ssim / len(val_loader)

        print(f"Epoch {epoch+1:3d} │ PSNR: {avg_val_psnr:7.4f} dB │ SSIM: {avg_val_ssim:.6f}")

        training_log.append({
            'epoch': epoch + 1,
            'val_psnr': avg_val_psnr,
            'val_ssim': avg_val_ssim,
            'learning_rate': scheduler.get_last_lr()[0]
        })

        log_df = pd.DataFrame(training_log)
        log_df.to_csv(log_file_path, index=False)
        scheduler.step()

        if avg_val_psnr > best_psnr:
            best_psnr = avg_val_psnr
            print(f"  ↑ New best PSNR: {best_psnr:.4f} dB. Saving...")

            model_state = model.state_dict()
            needs_stripping = any(key.startswith('_orig_mod.') for key in model_state.keys())

            if needs_stripping:
                final_model_state = {
                    key[len('_orig_mod.'):] if key.startswith('_orig_mod.') else key: value
                    for key, value in model_state.items()
                }
            else:
                final_model_state = model_state

            torch.save({
                'epoch': epoch,
                'model_state_dict': final_model_state,
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'best_psnr': best_psnr,
            }, CHECKPOINT_FILE)

    total_time = time.time() - start_time
    print(f"\n{'='*70}")
    print(f"✓ TRAINING COMPLETE")
    print(f"{'='*70}")
    print(f"Total time: {total_time/3600:.2f} hours")
    print(f"Best PSNR: {best_psnr:.4f} dB\n")

    # Plot results
    print("Generating training curves...")
    plt.figure(figsize=(14, 5))
    plt.subplot(1, 2, 1)
    plt.plot(log_df['epoch'], log_df['val_psnr'], marker='o', linewidth=2, markersize=3)
    plt.title('Validation PSNR vs. Epoch', fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('PSNR (dB)')
    plt.grid(True, alpha=0.3)

    plt.subplot(1, 2, 2)
    plt.plot(log_df['epoch'], log_df['val_ssim'], marker='o', linewidth=2, markersize=3, color='orange')
    plt.title('Validation SSIM vs. Epoch', fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('SSIM')
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(OUTPUTS_DIR / 'training_curves.png', dpi=150)
    print(f"✓ Curves saved to {OUTPUTS_DIR / 'training_curves.png'}")
    plt.show()

## 9. FP16 Quantization

In [ ]:
print("\n" + "="*70)
print("FP16 QUANTIZATION")
print("="*70)

quant_model = ESPCN(upscale_factor=UPSCALE_FACTOR).to(DEVICE)
checkpoint = torch.load(CHECKPOINT_FILE, map_location=DEVICE)

original_state_dict = checkpoint['model_state_dict']
new_state_dict = {}
needs_prefix_stripping = any(key.startswith('_orig_mod.') for key in original_state_dict.keys())
if needs_prefix_stripping:
    print("Stripping '_orig_mod.' prefix...")
    for key, value in original_state_dict.items():
        new_key = key[len('_orig_mod.'):] if key.startswith('_orig_mod.') else key
        new_state_dict[new_key] = value
else:
    new_state_dict = original_state_dict

quant_model.load_state_dict(new_state_dict)
quant_model.eval()
quant_model.half()

quantized_model_path = CHECKPOINTS_DIR / f"best_espcn_x{UPSCALE_FACTOR}_fp16.pth"
torch.save(quant_model.state_dict(), quantized_model_path)
print(f"✓ FP16 model saved: {quantized_model_path}\n")

## 10. ONNX Export

In [ ]:
print("\n" + "="*70)
print("ONNX EXPORT")
print("="*70)

EXPORT_FP16 = True

if EXPORT_FP16:
    print(f"\nExporting FP16 model...")
    onnx_model = ESPCN(upscale_factor=UPSCALE_FACTOR)
    checkpoint_path_fp16 = CHECKPOINTS_DIR / f"best_espcn_x{UPSCALE_FACTOR}_fp16.pth"
    onnx_model.load_state_dict(torch.load(checkpoint_path_fp16))
    onnx_model.eval().half().to(DEVICE)
    dummy_input = torch.randn(1, 3, 720, 1280, device=DEVICE).half()
    onnx_model_path = OUTPUTS_DIR / f"espcn_x{UPSCALE_FACTOR}_fp16.onnx"
else:
    print(f"\nExporting FP32 model...")
    onnx_model = ESPCN(upscale_factor=UPSCALE_FACTOR).to(DEVICE)
    checkpoint = torch.load(CHECKPOINT_FILE, map_location=DEVICE)

    original_state_dict = checkpoint['model_state_dict']
    new_state_dict = {}
    needs_prefix_stripping = any(key.startswith('_orig_mod.') for key in original_state_dict.keys())

    if needs_prefix_stripping:
        print("Stripping '_orig_mod.' prefix...")
        for key, value in original_state_dict.items():
            new_key = key[len('_orig_mod.'):] if key.startswith('_orig_mod.') else key
            new_state_dict[new_key] = value
    else:
        new_state_dict = original_state_dict

    onnx_model.load_state_dict(new_state_dict)
    onnx_model.eval()
    dummy_input = torch.randn(1, 3, 720, 1280, device=DEVICE)
    onnx_model_path = OUTPUTS_DIR / f"espcn_x{UPSCALE_FACTOR}.onnx"

torch.onnx.export(
    onnx_model, dummy_input, str(onnx_model_path),
    export_params=True, do_constant_folding=True,
    input_names=['input'], output_names=['output'],
    dynamo=True,
    dynamic_axes={'input': {2: 'height', 3: 'width'}, 'output': {2: 'height_out', 3: 'width_out'}},
    opset_version=17
)

print(f"✓ ONNX model exported: {onnx_model_path}\n")

## 11. Performance Benchmarking

In [ ]:
print("="*70)
print("BENCHMARKING")
print("="*70 + "\n")

# Test input
if EXPORT_FP16:
    test_image_np = np.random.randn(1, 3, 720, 1280).astype(np.float16)
else:
    test_image_np = np.random.randn(1, 3, 720, 1280).astype(np.float32)

test_image_torch = torch.from_numpy(test_image_np).to(DEVICE)
if EXPORT_FP16:
    test_image_torch = test_image_torch.half()

# PyTorch benchmark
print("Benchmarking PyTorch (100 iterations)...")
if DEVICE == "cuda":
    torch.cuda.synchronize()

with torch.no_grad():
    _ = onnx_model(test_image_torch)
    if DEVICE == "cuda":
        torch.cuda.synchronize()
    start_time = time.time()
    for _ in range(100):
        _ = onnx_model(test_image_torch)
    if DEVICE == "cuda":
        torch.cuda.synchronize()
    pytorch_time = (time.time() - start_time) / 100

print(f"✓ PyTorch: {pytorch_time * 1000:.4f} ms/image\n")

# ONNX benchmark
print("Benchmarking ONNX Runtime (100 iterations)...")
session = ort.InferenceSession(str(onnx_model_path), providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
ort_inputs = {session.get_inputs()[0].name: test_image_np}
_ = session.run(None, ort_inputs)
start_time = time.time()
for _ in range(100):
    _ = session.run(None, ort_inputs)
onnx_time = (time.time() - start_time) / 100

print(f"✓ ONNX Runtime: {onnx_time * 1000:.4f} ms/image\n")

# Results
print("="*70)
if pytorch_time > 0:
    if onnx_time < pytorch_time:
        speedup = (pytorch_time - onnx_time) / pytorch_time * 100
        print(f"✓ ONNX is ~{speedup:.1f}% faster than PyTorch")
    else:
        slowdown = (onnx_time - pytorch_time) / pytorch_time * 100
        print(f"⚠ ONNX is ~{slowdown:.1f}% slower than PyTorch")
print("="*70)